## Load and inspect the data

In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv(r"C:\Users\Acer\Internship\Task4\data\stock_details_5_years.csv", parse_dates=['Date'], low_memory=False)
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2018-11-29 00:00:00-05:00,43.829761,43.863354,42.639594,43.083508,167080000,0.00,0.0,AAPL
1,2018-11-29 00:00:00-05:00,104.769074,105.519257,103.534595,104.636131,28123200,0.00,0.0,MSFT
2,2018-11-29 00:00:00-05:00,54.176498,55.007500,54.099998,54.729000,31004000,0.00,0.0,GOOGL
3,2018-11-29 00:00:00-05:00,83.749496,84.499496,82.616501,83.678497,132264000,0.00,0.0,AMZN
4,2018-11-29 00:00:00-05:00,39.692784,40.064904,38.735195,39.037853,54917200,0.04,0.0,NVDA


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602962 entries, 0 to 602961
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          602962 non-null  object 
 1   Open          602962 non-null  float64
 2   High          602962 non-null  float64
 3   Low           602962 non-null  float64
 4   Close         602962 non-null  float64
 5   Volume        602962 non-null  int64  
 6   Dividends     602962 non-null  float64
 7   Stock Splits  602962 non-null  float64
 8   Company       602962 non-null  object 
dtypes: float64(6), int64(1), object(2)
memory usage: 41.4+ MB


## Cleaning the data

### Type Conversion

In [17]:
df['Company'] = df['Company'].astype('category')
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
df.sort_values(['Company', 'Date'], inplace=True)

### Missing Value Imputation

In [18]:
# Forward fill for each company
df[['Open', 'High', 'Low', "Close", 'Volume']] = (
    df.groupby('Company')[["Open", 'High', "Low", "Close", "Volume"]].transform(lambda x: x.ffill().bfill())
    )

C:\Users\Acer\AppData\Local\Temp\ipykernel_29508\2745858499.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Company')[["Open", 'High', "Low", "Close", "Volume"]].transform(lambda x: x.ffill().bfill())


In [19]:
# Dividends/Stock Splits are often zero - fill missing with 0
df[['Dividends', 'Stock Splits']] = df[['Dividends', 'Stock Splits']].fillna(0)

### Outlier Removal (IQR on 'Close' price)

In [20]:
def remove_outliears_iqr(group, column='Close'):
    Q1 = group[column].quantile(0.25)
    Q3 = group[column].quantile(0.75)
    IQR = Q3 - Q1
    mask = (group[column] >= Q1 - 1.5 * IQR) & (group[column] <= Q3 + 1.5 * IQR)
    return group[mask]

df = df.groupby('Company', group_keys=False).apply(remove_outliears_iqr)

C:\Users\Acer\AppData\Local\Temp\ipykernel_29508\2807364820.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('Company', group_keys=False).apply(remove_outliears_iqr)
C:\Users\Acer\AppData\Local\Temp\ipykernel_29508\2807364820.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Company', group_keys=False).apply(remove_outliears_iqr)


## Schema Normalization

### Multi-Index + Time Series Alignment

In [21]:
df.set_index(['Company', "Date"], inplace=True)
df.sort_index(inplace=True)

### Normalize Volume with Rolling

In [22]:
df['volumn_7d_avg'] = df.groupby(level=0)['Volume'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

C:\Users\Acer\AppData\Local\Temp\ipykernel_29508\2444584356.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['volumn_7d_avg'] = df.groupby(level=0)['Volume'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())


###  Create Return Column

In [23]:
df['return'] = df.groupby(level=0)['Close'].pct_change()

C:\Users\Acer\AppData\Local\Temp\ipykernel_29508\3957419500.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['return'] = df.groupby(level=0)['Close'].pct_change()


## Feature Engineering

### Polynomial & Interaction Terms

In [24]:
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# Reset index to work with raw columns
df_reset = df.reset_index()

# Select features for polynomial expansion
features = df_reset[["Open", "High", "Low", "Close"]]

# Generate polynomial and interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_features = poly.fit_transform(features)

# Use safe column names (with feature interactions)
poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(features.columns))

# Concatenate with original dataframe (exclude original columns to avoid duplication)
df_fe = pd.concat([df_reset.drop(columns=features.columns), poly_df], axis=1)

### Encode Company

In [25]:
# Encode company as categorical
df_fe["Company_code"] = df_fe["Company"].astype("category").cat.codes

### Export Cleaned Dataset

In [26]:
# Save cleaned data
df_fe.to_csv('C:/Users/Acer/Internship/Task4/Cleaned data/stock_prices_cleaned.csv', index=False)